#BASIC INFORMATION (ReadMe)



**1.  About Jupyter Notebook and Google Colab**
- Jupyter Notebook is an open-source web application that allows you to create and share documents that contain live code, equations, visualizations and text. Uses include: data cleaning and transformation, numerical simulation, statistical modeling, data visualization, machine learning etc.
- Google Colaboratory is a free Jupyter notebook environment that requires no setup and runs entirely in the cloud. 

**2.   About this lab**

- In this lab we will explore and use Melbourne housepricing dataset.  Each row in the dataset is a housing sale and columns are the features of the sold house/appartment.


- The goal is to predict the price of new sales. We will build two supervised models: regression models and artificial neural network. ML pipeline outlined in the theory part of the course will be followed. 


**3.  How to save own version of the Jupyter notebook**

Go to File and choose "Save a copy in Drive". It will be saved on your Google Drive.

**4.   How to run/execute cell of code**

Mark the cell,  press "Run" button on the left side of the cell or Ctrl+Shift tab through the code.


**5.  More info about dataset**
 - Data source: https://www.kaggle.com/anthonypino/melbourne-housing-market


 - Columns:
1. Suburb: Suburb
2. Address: Address
3. Rooms: Number of rooms
4. Price: Price in Australian dollars, AUD
5. Method: S - property sold; SP - property sold prior; PI - property passed in; PN - sold prior not disclosed; SN - sold not disclosed; NB - no bid; VB - vendor bid; W - withdrawn prior to auction; SA - sold after auction; SS - sold after auction price not disclosed. N/A - price or highest bid not available.
6. Type: br - bedroom(s); h - house,cottage,villa, semi,terrace; u - unit, duplex; t - townhouse; dev site - development site; o res - other residential.
7. SellerG: Real Estate Agent
8. Date: Date sold
9. Distance: Distance from CBD in Kilometres 
10. Regionname: General Region (West, North West, North, North east ...etc)
11. Propertycount: Number of properties that exist in the suburb
12. Bedroom2 : Scraped # of Bedrooms (from different source)
13. Bathroom: Number of Bathrooms
14. Car: Number of carspots
15. Landsize: Land Size in Metres
16. BuildingArea: Building Size in Metres
17. YearBuilt: Year the house was built
18. CouncilArea: Governing council for the area
19. Lattitude: Self explanitory
20. Longtitude: Self explanitory


#1) CLONE ENVIRONMENT & IMPORT LIBRARIES

##1.1. Get all the files
- Run the below code cell if the notebook is opened in Google Collab. It will clone the github repository to get all necessary files.

In [0]:
!git clone https://github.com/NordAxon/AI-For-Leaders.git

## 1.2 Import libraries
- ML with Python offers a great deal of libraries.
- Read about some of the most used: https://hackernoon.com/top-10-libraries-in-python-to-implement-machine-learning-12602cf5dc61

- Lets import all the libraries we need to run the code and perform the analysis. Run the below code cell.

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

import seaborn as sns

pd.set_option('display.max_columns', 100)
%load_ext autoreload
%autoreload 2

np.random.seed(1)

# 2) IMPORT RAW DATA

In [0]:
housing_df_original = pd.read_csv('AI-For-Leaders/data/melbourne-housing-market/Melbourne_housing.csv')
housing_df_original

In [0]:
# Size of the dataset (X, Y)
housing_df_original.shape

# 3) EXPLORE DATA & PRE-PROCESSING

##  3.1. Fill in empty/missing data

- In some of the columns we have NaN(Not a Number) values i.e. missing data.

- Many ML algorithms need data in all rows and columns so the NaNs have to be filled with something meaningful. E.g. "Landsize" will be filled with 0 to indicate that there is no land contained in the real estate sale. 



 


Let's first have a look at how many NaN values are present in each column. 
- Run the code cell below

In [0]:
print('Number of NaNs for every variable:')
print(pd.isnull(housing_df_original).sum())

Lets clean and fill/replace missing/odd values
- Remove rows where there is no price
- Fill NaN values in columns BuildingArea, Rooms, Landsize, Car, Bathroom, Bedroom2 with 0
- For column YearBuilt, fill with the mean value of that column since the house being built at year 0 seems unlikely

In [0]:
housing_df_no_nan = housing_df_original.copy()

# Remove all rows with no price data
housing_df_no_nan = housing_df_no_nan[pd.notnull(housing_df_no_nan['Price'])]

# Replace NaN values with 0 values in columns BuildingArea, Rooms, Landsize, Car, Bathroom, Bedroom2 
housing_df_no_nan['BuildingArea'].fillna(0.0, inplace=True)
housing_df_no_nan['Rooms'].fillna(0.0, inplace=True)
housing_df_no_nan['Landsize'].fillna(0.0, inplace=True)
housing_df_no_nan['Car'].fillna(0.0, inplace=True)
housing_df_no_nan['Bathroom'].fillna(0.0, inplace=True)
housing_df_no_nan['Bedroom2'].fillna(0.0, inplace=True)

# Replace NaN values in YearBuilt column with mean value of that column
housing_df_no_nan['YearBuilt'].fillna(housing_df_no_nan['YearBuilt'].mean(), inplace=True)

print('Number of NaNs for every variable:')
print(pd.isnull(housing_df_no_nan).sum())

##**3.2. Plot variable correlations and histograms**

- Let's have a look at how different variables relate to each other and how the data is distributed. 
- The goal is to get insight in our dataset, understand how different columns relate to each other and look for outliers.
- Let pick some columns(variables) we think might be good predictors of price and scatter plot those against each other and as histograms along the diagonal. In a real project we would have plotted all variables in many different ways, but we try to restrict the time spent on this by only picking six variables.


In [0]:
plot_str_arr = ['BuildingArea', 'Rooms', 'Landsize', 'Car', 'Bathroom', 'Price']
f, axarr = plt.subplots(2, len(plot_str_arr), figsize=(22,7))

i = 0
for plt_str in plot_str_arr:
    sns.regplot(x=plt_str, y="Price", data=housing_df_no_nan, ax=axarr[0,i])
    housing_df_no_nan[plt_str].hist(ax=axarr[1,i])
    i += 1
axarr[1,5].set_ylim([0,7000])
f.canvas.set_window_title('Scatter Plots and Histograms')

## 3.3. Remove outliers

- Outliers are datapoints located far from the other data points, these come with a risk of skewing the models and therefore we want to remove these. 

- In the plots above it seems like we have outliers in some of the columns. We will have a deeper look at some columns which seem to contain outliers. 

- Run the cell below to see a plot of BuildingArea vs Price where it is clear that we have some data points which are far away from the others. 

In [0]:
housing_df_no_nan.plot.scatter('BuildingArea', 'Price', title='Price [AUD] vs. Building Area [m^2]')

1.   Set clip-offs:
>*  Building area = 500 m^2 (all data points larger than 500 will be set to 500)
>*  Price= 4 000 000 AUD
2. Plot again! 

In [0]:
housing_clipped = housing_df_no_nan.copy()
housing_clipped['BuildingArea'] = housing_clipped['BuildingArea'].clip(0, 500)
housing_clipped['Price'] = housing_clipped['Price'].clip(0, 4e6)
housing_clipped.plot.scatter('BuildingArea', 'Price', title='Price [AUD] vs. Building Area [m^2]')

### ASSIGNMENT 1:

-  Plot "Landsize" (unit=m^2) vs "Price" in the empty cell below. 
- Does it look like there are some outliers? If so, what could be a resonable cut-off? (Hint: Have a look above at the statistical printouts for Landsize)
-  Clip the data set to remove Landsize outliers, i.e replace the value of variable landsize_max with a resonable number. Plot again!

In [0]:
# ENTER CODE HERE


# SOLUTION (filter out or clip outliers for Landsize)
housing_clipped.plot.scatter('Landsize', 'Price', title='Price vs. Landsize')
landsize_max = 800
housing_clipped_l = housing_clipped.copy()
housing_clipped_l['Landsize'] = housing_clipped['Landsize'].clip(0, landsize_max)
housing_clipped_l.plot.scatter('Landsize', 'Price', title='Price [AUD] vs. Landsize [m^2]')



## 3.4 Plot histograms for indicative columns
- Lets have a look more closely at the variables we think could be interesting by plotting seperate and larger histograms.

In [0]:

housing_clipped_l['BuildingArea'].hist(bins=40, figsize=(10,7))
plt.title('Histogram of Building Areas');
# TODO: Make it better

### ASSIGNMENT 2:

- Create a histogram of variable Price in the empty cell below.

In [0]:
# CODE HERE


# SOLUTION
housing_clipped_l['Price'].hist(bins=40, figsize=(10,7))
plt.title('Histogram of Price');

## 3.5 Price for different regions
- We are also hypothesising that the property location will have an impact on price. 
- Below a plot will be made to get a feel for how different locations might affect the price. 

In [0]:
plt.figure(figsize=(10,7))
housing_clipped_l.groupby('Regionname')['Price'].mean().plot.bar();

### ASSIGNMENT 3: 
- Plot Price for different CouncilArea

In [0]:
# CODE HERE

# SOLUTION
plt.figure(figsize=(10,7))
housing_clipped_l.groupby('CouncilArea')['Price'].mean().plot.bar();

# 4) BUILD/TRAIN MODEL: SIMPLE DIMENSIONAL LINEAR REGRESSION

- We are getting a feeling for what the data looks like, so now we might try a first model for predicting price. 
- The simple linear regression, one predictor variable (X) and one output variable (Y)  is very commonly used model. 

- We pick BuildingArea as a predictor to begin with since there seem to be a correlation between BuildingArea and Price according to our exploration. 

- The goal is to find all of the weights, $w_i$, in the following linear regression model. **Lookup slide XXXX for better explanation!!!!**


> $y = w_0 + w_1x_1$







## 4.1 Split dataset, pick algorithm, train model



In [0]:
# Set up input (x) and output (y) variables
x = housing_clipped_l[['BuildingArea']]
y = housing_clipped_l['Price']

# Split into test and train data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

# Set up and train simple linear regression model
regr = linear_model.LinearRegression()
regr.fit(x_train, y_train)

# Perform predictions on testset
y_pred_simple = regr.predict(x_test)

print (y_pred_simple)


### Plot results ###

# Prepare plotting figure and axes
plt.figure(figsize=(15,10));
plt.title('Simple Linear Regression Model');
plt.xlabel('BuildingArea [m^2]', fontsize=10)
plt.ylabel('Price [AUD]', fontsize=10)

# Plot training data in green color
plt.scatter(x_train.values, y_train.values, color='green', alpha=0.2)
# Plot test data in red color
plt.scatter(x_test.values, y_test.values, color='red', alpha=0.2)

# Plot the simple regression line based on test data in blue color
plt.plot(x_test, y_pred_simple, 'b')

print (regr.coef_)


## 4.2 Model evaluation
The error value will be compared to a baseline error which is the error if the prediction is just the mean of previous house values. 

In [0]:
# Evaluate Results

mean_absolute_error_simple = str(int((y_pred_simple - y_test).abs().mean()))
mean_baseline_error = str(int((y_train.mean() - y_test).abs().mean()))



print('* EVALUATION: Mean Absolute Error with Simple Linear Regression = ' + mean_absolute_error_simple)
print('* EVALUATION: Mean Baseline Error: ' + mean_baseline_error)

if mean_absolute_error_simple < mean_baseline_error:
    print ('\nMean Absolute Error with "Simple Linear Regression" is smaller then "Mean Baseline Error". Good!')

# 5) BUILD/TRAIN MODEL: MULTI-DIMENSIONAL LINEAR REGRESSION

- In order to increase the predictive power, i.e. to get a more accurate model, more information can be added to the model. 
- One way of doing that is by adding more input variables to the model. 
- Variables that could be tried are BuildingArea, Rooms, LandSize, Car. 


> $y = w_0 + w_1x_1 + w_2x_2 + \dots$


## 5.1 Split dataset, pick algorithm, train model

In [0]:
# Pick our in-variables i.e. features
features_input_list = ['BuildingArea', 'Rooms','Landsize', 'Car']
x = housing_clipped_l[features_input_list]
y = housing_clipped_l['Price']

# Split into test and train data
x_train_multidim, x_test_multidim, y_train_multidim, y_test_multidim = train_test_split(x, y, test_size=0.2)

# Set up and train regression model
regr_multidim = linear_model.LinearRegression()
regr_multidim.fit(x_train_multidim, y_train_multidim)

# Perform predictions
y_pred_multidim = regr_multidim.predict(x_test_multidim)

######### Plot results ##########

# Prepare plotting figure and axes
plt.figure(figsize=(15,10));
plt.title('Multidimensional Linear Regression Model');
plt.xlabel('BuildingArea [m^2]', fontsize=10)
plt.ylabel('Price [AUD]', fontsize=10)

# Plot training data in green color
plt.scatter(x_train_multidim['BuildingArea'].values, y_train_multidim.values, color='green', alpha=0.2)
# Plot test data in red color
plt.scatter(x_test_multidim['BuildingArea'].values, y_test_multidim.values, color='red', alpha=0.2)

# Plot the multidim regression line based on test data in black color
plt.plot(x_test_multidim['BuildingArea'], y_pred_multidim, 'k.')
#plt.plot(x_test_multidim['BuildingArea'], y_pred_simple, 'b')


# Print regression coefficients, w
print('***** Regression Coefficients: *****')
i=0
print(len(features_input_list))
while i<len(features_input_list):
    print(features_input_list[i] + ' = ' + str(regr_multidim.coef_[i]))
    i+=1


"""
print x_test['BuildingArea'].shape
print y_pred_multidim.shape
print x_test['BuildingArea']
print y_pred_multidim

# Transform data
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)
y_train_scaled = scaler.fit_transform(y_train.values.reshape(len(y_train),1))
y_test_scaled = scaler.transform(y_test.values.reshape(len(y_test),1))

print x_train_scaled
print y_train_scaled
"""

## 5.2 Model evaluation

In [0]:
mean_absolute_error_multidim = str(int((y_pred_multidim - y_test_multidim).abs().mean()))
print('* EVALUATION: Mean Absolute Error with Multi-Dimensional Linear Regression = ' + mean_absolute_error_multidim)
print('* EVALUATION: Mean Absolute Error with Simple Linear Regression = ' + mean_absolute_error_simple)

if mean_absolute_error_multidim < mean_absolute_error_simple:
    print ('\n Mean Absolute Error with "Multi-Dimensional Linear Regression" is smaller then with "Simple Linear Regression". \n Good, our regression model is improving!')

## ASSIGNMENT 4:
- Test your multidimensional model with different input parameters by picking them from the list of 20 available and compare results? 

>-->  **Hint**: Modify 2nd line of code in section 5.1, add/remove features in features_input_list. Rerun the model and evaluate!



# 6) BUILD/TRAIN MODEL: ARTIFICAL NEURAL NETWORK


## 6.1 Split dataset, pick algorithm, train & evaluate model

The code in the following cell transforms data, builds a neural network and evaluates results of predictions from the neural net.

Good reading about different steps with Keras:
https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

In [0]:
# from numpy.random import seed
# seed(1)
# Filter out the wanted columns

def run_neural_network(x, y, n_epochs=10, layer_list=[100,20]):
    x_train_nn, x_test_nn, y_train_nn, y_test_nn = train_test_split(x, y, train_size=0.5, test_size=0.5)

    # Scale the data
    scaler = MinMaxScaler()
    x_train_nn = scaler.fit_transform(x_train_nn)
    x_test_nn = scaler.transform(x_test_nn)
    y_train_nn = scaler.fit_transform(y_train_nn.values.reshape(len(y_train_nn),1))
    y_test_nn= scaler.transform(y_test_nn.values.reshape(len(y_test_nn),1))
   
    # Define the NN structure (with Keras): 3 layers with (100,20,1) neurons, train.shape[1] input variables
    model = Sequential()
    model.add(Dense(layer_list[0], input_dim=x_train_nn.shape[1], activation='relu'))
    for layer_size in layer_list[1:]:
        model.add(Dense(layer_size, activation='relu'))
    model.add(Dense(1, activation='linear'))
    
    # Plot model summary
    print(model.summary())
    
    # Compile model 
    model.compile(loss='mse', optimizer='adam', metrics=['mean_squared_error', 'mean_absolute_error'])

    # Train the model
    history = model.fit(x_train_nn, y_train_nn, epochs=n_epochs, verbose=0, validation_data=(x_test_nn, y_test_nn))

    # Make a prediction
    y_pred_nn = model.predict(x_test_nn)[:,0]
    
    # Show the inputs and predicted outputs
    y_pred_nn = scaler.inverse_transform(y_pred_nn.reshape(len(y_pred_nn),1))
    y_test_nn = scaler.inverse_transform(y_test_nn)

    # Evaluate results
    mean_error = (pd.Series(y_pred_nn[:,0]) - y_test_nn[:,0]).abs().mean()
    print('EVALUATION: Mean Absolute Test Error: ' + str(mean_error))

    # Plot error over training time
    plt.figure(figsize=(10,7))
    plt.plot(history.history['mean_squared_error'])
    plt.plot(history.history['val_mean_squared_error'])
    plt.title('Model Loss')
    plt.ylabel('Mean Squared Error')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
    

## 6.2 Run NN model:

In [0]:
# Pick in-variables i.e. features
features_input_list = ['BuildingArea', 'Rooms', 'Landsize', 'Car']
x = housing_clipped_l[features_input_list]
y = housing_clipped_l['Price']

# Call/execute neural network
run_neural_network(x,y, 10, [5, 2])

**ASSIGNMENT 5:**
*  Try different number of training epochs by adjusting the code above.
*   What happens with the loss for a higher amount of training epochs(time)?






## 6.3 Improving NN model
Adding more features:
- Let's look at the data to see what we can do with the data to create columns which are more easily readable for a ML algorithm and how we can provide more information from the data we have. 
- Lets try and make the predictive model as good as possible by adding more features such as location and age of property. 

### 6.3.1 Add feature: House age
- Using the build year of a house directly as a feature is not good since most values will be around 2000. 
- A small difference in feature might be a big difference in actual house value; e.g. a house built 2017 is probably a lot more valuable than a house build 2007, but that is still a small percentual difference. The percentual difference between the age of 1 year and 11 years on the other hand gives a large difference. 
- We also log-transform the age to make it a bit more convenient for machine learning algorithms. 

In [0]:
year_built = housing_clipped_l['YearBuilt'].clip(0, 2018)
house_ages = pd.Series(np.log((1 + (2018. - year_built))))
house_ages = house_ages.fillna(house_ages.mean())
house_ages.name = 'Age'
plt.figure()
house_ages.hist()
plt.title('Distribution After logarithm')
house_ages_df = pd.DataFrame(house_ages)
features_w_age = pd.concat([housing_clipped_l, house_ages_df], axis=1)

Run the model with house age added.

In [0]:
age_feature_list = ['BuildingArea', 'Rooms', 'Landsize', 'Car'] + ['Age'] # + list(one_hot_region.columns)

run_neural_network(features_w_age[age_feature_list], features_w_age['Price'])

### 6.3.2 Add feature: Regionname 
Add Regionname as a feature by transforming it to one-hot

In [0]:
# One-hot encoding of Region
one_hot_region = pd.get_dummies(housing_clipped_l.Regionname, prefix='Regionname')
region_feature = pd.concat([housing_clipped_l, one_hot_region], axis=1)
region_feature

Train network with Regionname added

In [0]:
age_feature_list = ['BuildingArea', 'Rooms', 'Landsize', 'Car'] + list(one_hot_region.columns)

run_neural_network(region_feature[age_feature_list], region_feature['Price'])

### ASSIGNMENT 5:
Try different number of training epochs by adjusting the code above.
-  What happens with the loss for a higher amount of training epochs(time)?


ANSWER HERE (doubleclick):



### ASSIGNMENT 6:
Try different sizes of the network by adjusting the code above. Try out the below network configurations and observe the mean absolute error for test and train data.The lists below are denoted such that the first entry represents the number of neurons in the first layer and so on. 

1.   Configuration 1: [5, 2]
2.   Configuration 2: [100, 20]  (increasing the number or neurons)
3.   Configuration 3: [1000, 100] (increasing the number or neurons even more)
4.   Configuration 4: [5, 5, 5, 2] (more layers, the term deep learning originates from using many layers)
5    Configuration 5: [500, 500, 200] (larger network)
6.   Configuration 6: Test your own configuration if you want to.  


--> **Hint:** Modify 7th line of code under section 6.2.

**6.1**. What are the results when adding more layers? <br>
**6.2**. What are the results when adding more neurons? <br>
**6.3** When is there a difference between train and test data in absolute error? 

ANSWER HERE (doubleclick):

### ASSIGNMENT 7: 
Trying adding more features  by adjusting the code above.
-  Which columns could be useful for providing more predictive power?

ANSWER HERE (doubleclick):

# *DISCUSSION QUESTIONS:*


1.   Which of the different models performed best? Why?
2.   Why does a well tuned neural network perform better than a linear regression model?
3.   What could be done to increase predictive power?
4.   Which additional data do you think would make a large differce in predictive power?
5.   What was the lowest mean square error you got?




ANSWER HERE (doubleclick):

1. 
2. 
3. 
4. 
5. 


